In [1]:
# Import packages

import pandas as pd
import numpy as np

In [2]:
import altair as alt

In [3]:
# Load Kenya MTF Data 

chunksize = 10

mtf_list = []
chunksize = 10
for chunk in pd.read_csv("section_l.csv", encoding='latin-1', chunksize = chunksize):
    mtf_list.append(chunk)
    
section_l = pd.concat(mtf_list, axis=0)
df = pd.DataFrame(data = section_l)

In [4]:
# Tools

import functools
def conjunction(*conditions):
    return functools.reduce(np.logical_and, conditions)

In [5]:
def catalyst(*args, **kwargs):
    font = "Times New Roman"
    labelFont = "Times New Roman" 
    sourceFont = "Times New Roman"
    # Axes
    axisColor = "#000000"
    gridColor = "#DEDDDD"
    # Colors
    main_palette = [   '#2980B9', #(light blue)
        '#003366', #(dark blue)
        '#FFAE19', #(gold)
        '#D68910', #(dark gold)
        '#E67E22', #(orange)
        '#D35400', #(dark orange)
        '#FF4500', #(red)
        '#FF6162', #(dark pink/red)
        '#438B28', #(green)
        '#45B39D', #(teal)
        '#99ADC1', #(light gray)
        '#212F3C', #(dark gray)
    ]
    return {
        "width": 800, 
        "height": 400, 
        "config": {
            "title": {
                "fontSize": 18,
                "font": font,
                "anchor": "center",
                "fontColor": "#000000",
            },
            "axisX": {
                "domain": True,
                "domainColor": axisColor,
                #"domainWidth": 1,
                "grid": False,
                "labelFont": labelFont,
                "labelFontSize": 14,
                #"labelAngle": 0,
                #"tickColor": axisColor,
                #"tickSize": 5, 
                "titleFont": font,
                "titleFontSize": 14,
                "titleFontWeight": "normal",
                #"titlePadding": 10, 
                "title": "X Axis Title (units)", 
                #"titleAngle":0
            },
            "axisY": {
                "domain": True,
                "grid": False,
                "gridColor": gridColor,
                "gridWidth": 1,
                "labelFont": labelFont,
                "labelFontSize": 14,
                "labelAngle": 0,
                "ticks": False, 
                "titleFont": font,
                "titleFontSize": 14,
                "titleFontWeight": "normal",
                "titlePadding": 10, 
                "title": "Y Axis Title (units)",
                #"titleY": -10, 
                #"titleX": 18,
                #"titleAngle": 0 #-90
            },
            "range": {
                "category": main_palette
            },
            "legend": {
                "labelFont": labelFont,
                "labelFontSize": 14,
                "symbolType": "square", 
                "symbolSize": 100, 
                "titleFont": font,
                "titleFontSize": 14,
                "titleFontWeight": "normal",
                "title": "Legend", 
                "orient": "right", 
                "offset": 0, 
            },
            "tooltip":{
                "font": "Open Sans"
            },
            "view": {
                "stroke": "transparent", 
        ### MARKS CONFIGURATIONS ###
           "text": {
               "font": "Times New Roman",
               "color": main_palette,
               "fontSize": 14,
               "align": "right",
               "fontWeight": 400,
               "size": 14,
           }, 
           "bar": {
                "size": 40,
                "binSpacing": 1,
                "continuousBandSize": 30,
                "discreteBandSize": 30,
                "fill": main_palette,
                "stroke": False,
           },
            }
        }
    }

In [6]:
alt.themes.register('catalyst', catalyst)
alt.themes.enable('catalyst');

In [36]:
data = pd.DataFrame({
    'x': range(12),
    'color': [   '#2980B9', #(light blue)
        '#003366', #(dark blue)
        '#FFAE19', #(gold)
        '#D68910', #(dark gold)
        '#E67E22', #(orange)
        '#D35400', #(dark orange)
        '#FF4500', #(red)
        '#FF6162', #(dark pink/red)
        '#438B28', #(green)
        '#45B39D', #(teal)
        '#99ADC1', #(light gray)
        '#212F3C', #(dark gray)
    ]
})

alt.Chart(data).mark_point(
    filled=True,
    size=200
).encode(
    x='x',
    color=alt.Color('color', scale=None)
).properties(
    height = 100
)

alt.Chart(...)

In [11]:
# l15

# L.2
alt.data_transformers.disable_max_rows()
source1 = section_l[section_l[['l_l_15']].notnull().all(1)]

c_1 = source1['l_l_15'] != 'NaN'
c_2 = source1['elc_aggr_tier'] != 'NaN'

source = source1[conjunction(c_1,c_2)]
df = source[['elc_aggr_tier','l_l_15', 'locality_ur']]

chart = alt.Chart(df).transform_joinaggregate(
    total='count(*)'
).transform_calculate(
    pct='1 / datum.total'                
).mark_rect(color = '#2980B9'
).encode(
    x = alt.X('l_l_15', 
              bin=alt.Bin(extent=[1, 6000], step=200), 
              scale = alt.Scale(domain=(1, 6000),clamp = True),
              axis = alt.Axis(title = "Kenyan Shillings ")),
    y= alt.Y('sum(pct):Q', axis=alt.Axis(format='%'), title = "Percentage"),
    tooltip = [alt.Tooltip('l_l_15:N', title = "Response (Binned)"), 
               alt.Tooltip('sum(pct):Q', format = '.2%', title = "")]
).properties(
    width = 800,
    height = 500,
    title = "Amount Spent on Mobile Phone Top-up "
)

# Tier filter
tiers = [None, 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_labels = ['All Tiers', 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_dropdown=alt.binding_select(options=tiers, labels = tier_labels, name = "Electricity Tier  ")
tier_select=alt.selection_single(fields=['elc_aggr_tier'],
                                bind=tier_dropdown)

locality_options = [None, 'Urban', 'Rural']
locality_labels = ['National', 'Urban', 'Rural']

locality_dropdown=alt.binding_select(
    options= locality_options, 
    labels = locality_labels, 
    name = "Locality Breakdown  ")
locality_select=alt.selection_single(
    fields=['locality_ur'],
    bind=locality_dropdown)

# charts

chart = (chart).add_selection(
    tier_select
).transform_filter(
    tier_select 
).add_selection(
    locality_select
).transform_filter(
    locality_select
)

chart = chart.properties(
    title={
      "text": ["Amount Spent on Mobile Phone Top-Up per Month"],
      "color": "black",
        "font": 'Times New Roman'
    }
)

final = alt.concat(chart,
    title=alt.TitleParams(
        ['', '',  
         'This chart includes responses from 87.10% of households in the Kenya MTF survey sample (total size = 4,590).',
         'Use the dropdown filters to interact with the data.'],
        baseline='bottom',
        orient='bottom',
        anchor='start',
        font = 'Times New Roman',
        fontWeight='normal',
        fontStyle = "italic",
        fontSize=15)
)

final = final.configure(font = 'Times New Roman')
final

alt.ConcatChart(...)

In [38]:
df.shape[0] / 4590

0.8710239651416122

In [12]:
final.save('Amount Spent on Mobile Phone Top-Up per Month.html')

In [13]:
final.save('Amount Spent on Mobile Phone Top-Up per Month.json')

In [14]:
# L.2

source1 = section_l[section_l[['l_l_17']].notnull().all(1)]

c_1 = source1['l_l_17'] != 'NaN'
c_2 = source1['elc_aggr_tier'] != 'NaN'

source = source1[conjunction(c_1,c_2)]
df = source[['elc_aggr_tier','l_l_17', 'locality_ur']]

# chart
chart = alt.Chart(df).transform_joinaggregate(
    total='count(*)'
).transform_calculate(
    pct='1 / datum.total'
).mark_rect(color = '#2980B9'
).encode(
    x = alt.X('l_l_17', 
              bin=alt.Bin(extent=[1, 25000], step=1000), 
              scale = alt.Scale(domain=(1, 25000),clamp = True),
              axis = alt.Axis(title = "Kenyan Shillings")),
    y= alt.Y('sum(pct):Q', axis=alt.Axis(format='%'), title = " "),
    tooltip = [alt.Tooltip('l_l_17:N', title = "Response (Binned)"), 
               alt.Tooltip('sum(pct):Q', format = '.2%', title = "Percentage")]
).properties(
    width = 800,
    height = 500,
    title = "Amount Spent on House Rent  "
)

# Locality Filter
locality_options = [None, 'Urban', 'Rural']
locality_labels = ['National', 'Urban', 'Rural']

locality_dropdown=alt.binding_select(
    options= locality_options, 
    labels = locality_labels, 
    name = "Locality Breakdown  ")
locality_select=alt.selection_single(
    fields=['locality_ur'],
    bind=locality_dropdown)

# Tier Filter
tiers = [None, 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_labels = ['All Tiers', 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_dropdown=alt.binding_select(options=tiers, 
                                 labels = tier_labels, 
                                 name = "Electricity Tier  ")
tier_select=alt.selection_single(fields=['elc_aggr_tier'],
                                bind=tier_dropdown)

# Primary cooking fuel

cooking_options = [None, 'Firewood', 'Charcoal', 'Kerosene', 'Not applicable',
                   'Biomass Briquette', 'Electricity', 'Biogas', 'Coal/lignite']

cooking_labels = ['All responses', 'Firewood', 'Charcoal', 'Kerosene', 'Not applicable',
                  'Biomass Briquette', 'Electricity', 'Biogas', 'Coal/lignite']

cooking_dropdown = alt.binding_select(
    options = cooking_options, 
    labels = cooking_labels, 
    name = "Primary source of cooking fuel  ")

cooking_select=alt.selection_single(
    fields=['Primary_1st_fuel'],
    bind=cooking_dropdown)

# Primary lighting filter 

lighting_options = [None, 'National grid',
                   'Rechargeable Battery',
                   'No electricity', 
                   'Solar Lantern/Lighting System',
                   'Solar Home System',
                   'Generator (Connecting one or more households)',
                   'Other']

lighting_labels = ['All responses', 'National grid',
                   'Rechargeable Battery',
                   'No electricity', 
                   'Solar Lantern/Lighting System',
                   'Solar Home System',
                   'Generator',
                   'Other']

lighting_dropdown = alt.binding_select(
    options = lighting_options, 
    labels = lighting_labels, 
    name = "Primary source of electricity  ")

lighting_select=alt.selection_single(
    fields=['c_c_159'],
    bind=lighting_dropdown)

chart = (chart).add_selection(
    tier_select
).transform_filter(
    tier_select 
).add_selection(
    locality_select
).transform_filter(
    locality_select
)

chart = chart.properties(
    title={
      "text": ["Amount Spent on House Rent per Month"],
      "color": "black",
        "font": 'Times New Roman'
    }
)

final = alt.concat(chart,
    title=alt.TitleParams(
        ['', '',  
         'This chart includes responses from 24.18% of households in the Kenya MTF survey sample (total size = 4,590).',
         'Use the dropdown filters to interact with the data.'],
        baseline='bottom',
        orient='bottom',
        anchor='start',
        font = 'Times New Roman',
        fontWeight='normal',
        fontStyle = "italic",
        fontSize=15)
)

final = final.configure(font = 'Times New Roman')
final

alt.ConcatChart(...)

In [207]:
df.shape[0] / 4590

0.24183006535947713

In [15]:
final.save('Amount Spent on House Rent per Month.html')

In [16]:
final.save('Amount Spent on House Rent per Month.json')

In [18]:
# Load Kenya MTF Data 

chunksize = 10

mtf_list = []
chunksize = 10
for chunk in pd.read_csv("module2.csv", encoding='latin-1', chunksize = chunksize):
    mtf_list.append(chunk)
    
module2 = pd.concat(mtf_list, axis=0)
module2 = pd.DataFrame(data = module2)

In [45]:
# In the last 12 months, how many HOURS did you use [LAMP/CANDLE]?

alt.data_transformers.disable_max_rows()
source1 = module2[module2[['num_days']].notnull().all(1)]

c_1 = source1['num_days'] != 'NaN'
c_2 = source1['elc_aggr_tier'] != 'NaN'

source = source1[conjunction(c_1,c_2)]
df = source[['elc_aggr_tier','num_days', 'locality','lamp_type']]

# Locality filter    
locality_options = [None, 'Urban', 'Rural', 'Peri-urban']
locality_labels = ['National', 'Urban', 'Rural', 'Peri-Urban']
locality_dropdown=alt.binding_select(
    options= locality_options, 
    labels = locality_labels, 
    name = "Locality Breakdown  ")
locality_select=alt.selection_single(
    fields=['locality'],
    bind=locality_dropdown)

# Tier filter
tiers = [None, 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_labels = ['All Tiers', 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_dropdown=alt.binding_select(options=tiers, 
                                 labels = tier_labels, 
                                 name = "Electricity Tier  ")
tier_select=alt.selection_single(fields=['elc_aggr_tier'],
                                bind=tier_dropdown)

# Source filter
sources = [None, 'Candles',  'Hurricane Lamps', 'Open Wick Lamps', 'Pressurized Lamps']
source_labels = ['All Sources', 'Candles', 'Hurricane Lamps', 'Open Wick Lamps', 'Pressurized Lamps']
source_dropdown=alt.binding_select(options=sources, 
                                 labels = source_labels, 
                                 name = "Lamp/Candle Source  ")
source_select=alt.selection_single(fields=['lamp_type'],
                                bind=source_dropdown)

# Chart
f8 = alt.Chart(df).transform_joinaggregate(
    total='count(*)'
).transform_calculate(
    pct='1 / datum.total'                              
).mark_bar(color = '#2980B9').encode(
    alt.X("num_days", 
          bin=alt.Bin(extent=[1, 31], step=1), 
          scale = alt.Scale(domain=(1,31),clamp = True),
          axis = alt.Axis(title = "Days per month ")),
    y= alt.Y('sum(pct):Q', axis=alt.Axis(format='%'), title = " "),
    tooltip = [alt.Tooltip('num_days:N', title = "Response (Binned)"), 
               alt.Tooltip('sum(pct):Q', format = '.2%', title = "Percentage")]
).transform_bin(
    "mbin",
    field="m",
    bin=alt.Bin(maxbins=10)
).add_selection(
    locality_select
).transform_filter(
    locality_select
).add_selection(
    source_select
).transform_filter(
    source_select
).add_selection(
    tier_select
).transform_filter(
    tier_select 
).properties(
    height = 500,
    width = 800)

chart = f8.properties(
    title={
      "text": ["How many days do you use your lamp or candle per month?"],
      "color": "black",
        "font": 'Times New Roman'
    }
)

final = alt.concat(chart,
    title=alt.TitleParams(
        ['', '',  
         'This chart includes responses from 60.46% of households in the Kenya MTF survey sample (total size = 4,590).',
         'Use the dropdown filters to interact with the data.'],
        baseline='bottom',
        orient='bottom',
        anchor='start',
        font = 'Times New Roman',
        fontWeight='normal',
        fontStyle = "italic",
        fontSize=15)
)

final = final.configure(font = 'Times New Roman')
final

alt.ConcatChart(...)

In [46]:
df.shape[0] / 4590

0.6045751633986928

In [47]:
final.save('Days used Lamp or Candle per Month.html')

In [48]:
final.save('Days used Lamp or Candle per Month.json')

In [49]:
# In the last 12 months, how many HOURS did you use [LAMP/CANDLE]?

alt.data_transformers.disable_max_rows()
source1 = module2[module2[['num_hours']].notnull().all(1)]

c_1 = source1['num_hours'] != 'NaN'
c_2 = source1['elc_aggr_tier'] != 'NaN'

source = source1[conjunction(c_1,c_2)]
df = source[['elc_aggr_tier','num_hours', 'locality','lamp_type']]

# Locality filter    
locality_options = [None, 'Urban', 'Rural', 'Peri-urban']
locality_labels = ['National', 'Urban', 'Rural', 'Peri-Urban']
locality_dropdown=alt.binding_select(
    options= locality_options, 
    labels = locality_labels, 
    name = "Locality Breakdown  ")
locality_select=alt.selection_single(
    fields=['locality'],
    bind=locality_dropdown)

# Tier filter
tiers = [None, 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_labels = ['All Tiers', 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_dropdown=alt.binding_select(options=tiers, 
                                 labels = tier_labels, 
                                 name = "Electricity Tier  ")
tier_select=alt.selection_single(fields=['elc_aggr_tier'],
                                bind=tier_dropdown)

# Source filter
sources = [None, 'Candles',  'Hurricane Lamps', 'Open Wick Lamps', 'Pressurized Lamps']
source_labels = ['All Sources', 'Candles', 'Hurricane Lamps', 'Open Wick Lamps', 'Pressurized Lamps']
source_dropdown=alt.binding_select(options=sources, 
                                 labels = source_labels, 
                                 name = "Lamp/Candle Source  ")
source_select=alt.selection_single(fields=['lamp_type'],
                                bind=source_dropdown)

# Chart
f9 = alt.Chart(df).transform_joinaggregate(
    total='count(*)'
).transform_calculate(
    pct='1 / datum.total'                              
).mark_bar(color = '#2980B9').encode(
    alt.X("num_hours", 
          bin=alt.Bin(extent=[0, 24], step=2), 
          scale = alt.Scale(domain=(0,24),clamp = True),
          axis = alt.Axis(title = "Hours per day ")),
    y= alt.Y('sum(pct):Q', axis=alt.Axis(format='%'), title = " "),
    tooltip = [alt.Tooltip('num_hours:N', title = "Response (Binned)"), 
               alt.Tooltip('sum(pct):Q', format = '.2%', title = "Percentage")]
).transform_bin(
    "mbin",
    field="m",
    bin=alt.Bin(maxbins=10)
).add_selection(
    locality_select
).transform_filter(
    locality_select
).add_selection(
    source_select
).transform_filter(
    source_select
).add_selection(
    tier_select
).transform_filter(
    tier_select 
).properties(
    height = 500,
    width = 800)

chart = f9.properties(
    title={
      "text": ["How many hours do you use your lamp or candle each day?"],
      "color": "black",
        "font": 'Times New Roman'
    }
)

final = alt.concat(chart,
    title=alt.TitleParams(
        ['', '',  
         'This chart includes responses from 60.15% of households in the Kenya MTF survey sample (total size = 4,590).',
         'Use the dropdown filters to interact with the data.'],
        baseline='bottom',
        orient='bottom',
        anchor='start',
        font = 'Times New Roman',
        fontWeight='normal',
        fontStyle = "italic",
        fontSize=15)
)

final = final.configure(font = 'Times New Roman')
final

alt.ConcatChart(...)

In [50]:
final.save('Hours used Lamp or Candle per Day.html')

In [51]:
final.save('Hours used Lamp or Candle per Day.json')

In [82]:
df.shape[0] / 4590

0.601525054466231

In [19]:
# In the last 12 months, how much did you SPEND WEEKLY?
source1 = module2[module2[['spent_weekly']].notnull().all(1)]
alt.data_transformers.disable_max_rows()

c_1 = source1['spent_weekly'] != 'NaN'
c_2 = source1['elc_aggr_tier'] != 'NaN'

source = source1[conjunction(c_1,c_2)]
df = source[['elc_aggr_tier','spent_weekly', 'locality','lamp_type']]

# Locality filter    
locality_options = [None, 'Urban', 'Rural', 'Peri-urban']
locality_labels = ['National', 'Urban', 'Rural', 'Peri-Urban']
locality_dropdown=alt.binding_select(
    options= locality_options, 
    labels = locality_labels, 
    name = "Locality Breakdown  ")
locality_select=alt.selection_single(
    fields=['locality'],
    bind=locality_dropdown)

# Tier filter
tiers = [None, 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_labels = ['All Tiers', 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_dropdown=alt.binding_select(options=tiers, 
                                 labels = tier_labels, 
                                 name = "Electricity Tier  ")
tier_select=alt.selection_single(fields=['elc_aggr_tier'],
                                bind=tier_dropdown)

# Source filter
sources = [None, 'Candles', 'Hurricane Lamps', 'Open Wick Lamps', 'Pressurized Lamps']
source_labels = ['All Sources', 'Candles', 'Hurricane Lamps', 'Open Wick Lamps', 'Pressurized Lamps']
source_dropdown=alt.binding_select(options=sources, 
                                 labels = source_labels, 
                                 name = "Lamp/Candle Source  ")
source_select=alt.selection_single(fields=['lamp_type'],
                                bind=source_dropdown)

# Chart
f11 = alt.Chart(df).transform_joinaggregate(
    total='count(*)'
).transform_calculate(
    pct='1 / datum.total'                
).mark_bar(color = '#2980B9'
).encode(
    alt.X("spent_weekly", 
          bin=alt.Bin(extent=[1, 900], step=50), 
          scale = alt.Scale(domain=(1,900),clamp = True),
          axis = alt.Axis(title = "Kenyan Shillings ")),
    y= alt.Y('sum(pct):Q', axis=alt.Axis(format='%'), title = " "),
    tooltip = [alt.Tooltip('spent_weekly:N', title = "Response (Binned)"), 
               alt.Tooltip('sum(pct):Q', format = '.2%', title = "Percentage")]
).transform_bin(
    "mbin",
    field="m",
    bin=alt.Bin(maxbins=25)
).add_selection(
    locality_select
).transform_filter(
    locality_select
).add_selection(
    source_select
).transform_filter(
    source_select
    ).add_selection(
    tier_select
).transform_filter(
    tier_select 
).properties(
    height = 500,
    width = 800)

chart = f11.properties(
    title={
      "text": ["How much did you spend weekly on the lamp or candle?"],
      "color": "black",
        "font": 'Times New Roman'
    }
)

final = alt.concat(chart,
    title=alt.TitleParams(
        ['', '',  
         'This chart includes responses from 59.22% of households in the Kenya MTF survey sample (total size = 4,590).',
         'Use the dropdown filters to interact with the data.'],
        baseline='bottom',
        orient='bottom',
        anchor='start',
        font = 'Times New Roman',
        fontWeight='normal',
        fontStyle = "italic",
        fontSize=15)
)

final = final.configure(font = 'Times New Roman')
final

alt.ConcatChart(...)

In [90]:
df.shape[0] / 4590

0.592156862745098

In [20]:
final.save('Weekly Expenditure on Lamps or Candles.html')

In [21]:
final.save('Weekly Expenditure on Lamps or Candles.json')

In [ ]:
# i_5_cost, i_41_a, k6k7_fuelwood, wtp_d6, wtp_d7, wtp_e7, wtp_e8

In [22]:
# Load Kenya MTF Data 

chunksize = 10

mtf_list = []
chunksize = 10
for chunk in pd.read_csv("cooking_final.csv", encoding='latin-1', chunksize = chunksize):
    mtf_list.append(chunk)
    
cooking_final = pd.concat(mtf_list, axis=0)
cooking_final = pd.DataFrame(data = cooking_final)

In [23]:
# I.41: "How much do you spend on the 1ST [FUEL TYPE] for this stove in the last month/in a typical month when you use the stove? 

alt.data_transformers.disable_max_rows()
source = cooking_final[cooking_final[['i_i_41a','i_18_a_1st_fuel', 'i_3_label']].notnull().all(1)]

c_1 = source['elc_aggr_tier'] != 'NaN'
c_2 = source['i_i_41a'] != 'NaN'

source = source[conjunction(c_1,c_2)]
df = source[['elc_aggr_tier', 'locality', 'i_i_41a', 'i_18_a_1st_fuel','i_3_label']]

cooking_fuels = alt.Chart(df).transform_joinaggregate(
    total='count(*)'
).transform_calculate(
    pct='1 / datum.total'                
).mark_rect(color = '#2980B9').encode(
    x = alt.X('i_i_41a', 
              bin=alt.Bin(extent=[1, 5000], step=200), 
              scale = alt.Scale(domain=(1,5000),clamp = True),
              axis = alt.Axis(title = "Kenyan Shillings ")),
    y= alt.Y('sum(pct):Q', axis=alt.Axis(format='%'), title = " "),
    tooltip = [alt.Tooltip('i_i_41a:N', title = "Response (Binned)"), 
               alt.Tooltip('sum(pct):Q', format = '.2%', title = "Percentage")]
).properties(
    width = 800,
    height = 500,
    title = " "
)

# Locality filter 
locality_options = [None, 'Urban', 'Rural']
locality_labels = ['National', 'Urban', 'Rural']
locality_dropdown= alt.binding_select(
    options= locality_options, 
    labels = locality_labels, 
    name = "Locality Breakdown  ")
locality_select= alt.selection_single(
    fields=["locality"],
    bind=locality_dropdown)

# Tier filter
tiers = [None, 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_labels = ['All Tiers', 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_dropdown=alt.binding_select(options=tiers, labels = tier_labels, name = "Electricity Tier  ")
tier_select=alt.selection_single(fields=['elc_aggr_tier'],
                                bind=tier_dropdown)

# Stove filter
sources = [None,'Three stone/Open fire','Manufactured stove', 'Other Self-Built Stove']
source_labels = ['All Stove Types','Three stone/Open fire', 'Manufactured Stove', 'Other Self-Built Stove']
source_dropdown=alt.binding_select(options=sources, 
                                  labels = source_labels, 
                                  name = "Stove Type  ")
source_select=alt.selection_single(fields=['i_3_label'],
                                bind=source_dropdown)

# Fuel filter - exclude missing values like processed biomass, ethanol, coal briquette
fuels = [None, 'Kerosene', 'Coal/lignite', 'Charcoal', 'Crop Residue/Plant Biomass', 'Biogas', 'LPG/cooking gas', 'Electricity']
fuel_labels = ['All Fuel Types', 'Kerosene', 'Coal/lignite', 'Charcoal', 'Crop Residue', 'Biogas', 'LPG/cooking gas', 'Electricity']
fuel_dropdown=alt.binding_select(options = fuels, 
                                  labels = fuel_labels, 
                                  name = "Fuel Type  ")
fuel_select=alt.selection_single(fields=['i_18_a_1st_fuel'],
                                bind=fuel_dropdown)

i_41_a = cooking_fuels.add_selection(
    tier_select
).transform_filter(
    tier_select 
).add_selection(
    source_select
).transform_filter(
    source_select
).add_selection(
    fuel_select
).transform_filter(
    fuel_select
).add_selection(
    locality_select
).transform_filter(
    locality_select
)

chart = i_41_a.properties(
    title={
      "text": ["How much do you spend on fuel for stoves in a typical month?"],
      "color": "black",
        "font": 'Times New Roman'
    }
)

final = alt.concat(chart,
    title=alt.TitleParams(
        ['', '',  
         'This chart includes responses from 96.74% of households in the Kenya MTF survey sample (total size = 4,590).',
         'Use the dropdown filters to interact with the data.'],
        baseline='bottom',
        orient='bottom',
        anchor='start',
        font = 'Times New Roman',
        fontWeight='normal',
        fontStyle = "italic",
        fontSize=15)
)

final = final.configure(font = 'Times New Roman')
final

alt.ConcatChart(...)

In [24]:
final.save('Amount Spent on Fuel for Stoves.html')

In [25]:
final.save('Amount Spent on Fuel for Stoves.json')

In [106]:
df.shape[0]/cooking_final.shape[0]

0.9674166020170675

In [26]:
# I.5: How much did you pay for the cookstove (if you paid for one)?

alt.data_transformers.disable_max_rows()
source = cooking_final[cooking_final[['i_5_cost']].notnull().all(1)]

c_1 = source['elc_aggr_tier'] != 'NaN'
c_2 = source['i_5_cost'] != 'NaN'

source = source[conjunction(c_1,c_2)]
df = source[['elc_aggr_tier', 'locality', 'i_5_cost', 'i_18_b_2nd_fuel', 'i_18_a_1st_fuel', 'i_3_label']]

cooking_fuels = alt.Chart(df).transform_joinaggregate(
    total='count(*)'
).transform_calculate(
    pct='1 / datum.total'                
).mark_rect(color = '#2980B9').encode(
    x = alt.X('i_5_cost', 
              bin=alt.Bin(extent=[1, 15000], step=1000), 
              scale = alt.Scale(domain=(1,15000),clamp = True),
              axis = alt.Axis(title = "Kenyan Shillings ")),
    y= alt.Y('sum(pct):Q', axis=alt.Axis(format='%'), title = ""),
    tooltip = [alt.Tooltip('i_5_cost:N', title = "Response (Binned)"), 
               alt.Tooltip('sum(pct):Q', format = '.2%', title = "Percentage")]
).properties(
    width = 800,
    height = 500,
    title = " "
)

#Tier filter
tiers = [None, 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_labels = ['All Tiers', 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_dropdown=alt.binding_select(options=tiers, labels = tier_labels, name = "Electricity Tier  ")
tier_select=alt.selection_single(fields=['elc_aggr_tier'],
                                bind=tier_dropdown)

# Stove filter - not relevant, b/c manufactured stove is the only one purchased out of all options.
sources = [None,'Manufactured stove', 'Other Self-Built Stove']
source_labels = ['All Stove Types', 'Manufactured Stove', 'Other Self-Built Stove']
source_dropdown=alt.binding_select(options=sources, 
                                  labels = source_labels, 
                                  name = "Stove Type  ")
source_select=alt.selection_single(fields=['i_3_label'],
                                bind=source_dropdown)

# Primary Fuel filter - exclude missing values like processed biomass, ethanol, coal briquette
fuels = [None, 'Kerosene', 'Coal/lignite', 'Charcoal', 'Crop Residue/Plant Biomass', 'Saw Dust', 'Biogas', 'LPG/cooking gas', 'Electricity']
fuel_labels = ['All Fuel Types', 'Kerosene', 'Coal/lignite', 'Charcoal', 'Crop Residue', 'Saw Dust', 'Biogas', 'LPG/cooking gas', 'Electricity']
fuel_dropdown=alt.binding_select(options = fuels, 
                                  labels = fuel_labels, 
                                  name = "Primary Fuel Type  ")
fuel_select=alt.selection_single(fields=['i_18_a_1st_fuel'],
                                bind=fuel_dropdown)

# Secondary Fuel filter - exclude missing values like processed biomass, ethanol, coal briquette
sfuels = [None, 'Kerosene', 'Coal/lignite', 'Charcoal', 'Crop Residue/Plant Biomass', 'Saw Dust', 'LPG/cooking gas']
sfuel_labels = ['All Fuel Types', 'Kerosene', 'Coal/lignite', 'Charcoal', 'Crop Residue', 'Saw Dust', 'LPG/cooking gas']
sfuel_dropdown=alt.binding_select(options = sfuels, 
                                  labels = sfuel_labels, 
                                  name = "Secondary Fuel Type  ")
sfuel_select=alt.selection_single(fields=['i_18_b_2nd_fuel'],
                                bind=sfuel_dropdown)

i_5_cost = cooking_fuels.add_selection(
    tier_select
).transform_filter(
    tier_select 
).add_selection(
    sfuel_select
).transform_filter(
    sfuel_select
).add_selection(
    fuel_select
).transform_filter(
    fuel_select
)

chart = i_5_cost.properties(
    title={
      "text": ["How much did you pay for your cookstove?"],
      "color": "black",
        "font": 'Times New Roman'
    }
)

final = alt.concat(chart,
    title=alt.TitleParams(
        ['', '',  
         'This chart includes responses from 44.88% of households in the Kenya MTF survey sample (total size = 4,590).',
         'Use the dropdown filters to interact with the data.'],
        baseline='bottom',
        orient='bottom',
        anchor='start',
        font = 'Times New Roman',
        fontWeight='normal',
        fontStyle = "italic",
        fontSize=15)
)

final = final.configure(font = 'Times New Roman')
final

alt.ConcatChart(...)

In [118]:
df.shape[0] / cooking_final.shape[0]

0.44879751745539176

In [27]:
final.save('Amount Paid For Cookstoves.html')

In [28]:
final.save('Amount Paid For Cookstoves.json')

In [62]:
# Load Kenya MTF Data 

chunksize = 10

mtf_list = []
chunksize = 10
for chunk in pd.read_csv("k_cooking.csv", encoding='latin-1', chunksize = chunksize):
    mtf_list.append(chunk)
    
k_cooking = pd.concat(mtf_list, axis=0)
df = pd.DataFrame(data = k_cooking)

In [144]:
df['locality_ur']

0       Rural
1       Rural
2       Rural
3       Rural
4       Urban
        ...  
5151    Urban
5152    Urban
5153    Urban
5154    Urban
5155    Urban
Name: locality_ur, Length: 5156, dtype: object

In [145]:
locality_options = [None, 'Urban', 'Rural']
locality_labels = ['National', 'Urban', 'Rural']

locality_dropdown=alt.binding_select(
    options= locality_options, 
    labels = locality_labels, 
    name = "Locality Breakdown  ")
locality_select=alt.selection_single(
    fields=['locality_ur.x'],
    bind = locality_dropdown)

In [63]:
alt.data_transformers.disable_max_rows()
source = df[df[['k_k_fw_stove_6', 'k_k_fw_stove_7','locality_ur']].notnull().all(1)]

c_1 = source['k_k_fw_stove_6'] != 'NaN'
c_2 = source['k_k_fw_stove_7'] != 'NaN'

source = source[conjunction(c_1,c_2)]
df = source[['elc_aggr_tier','k_k_fw_stove_6', 'k_k_fw_stove_7', 'locality_ur']]

# Configure common options
base = alt.Chart(df).transform_aggregate(
    cookstove_count='count()',
    groupby=['k_k_fw_stove_6', 'k_k_fw_stove_7']
).encode(
    alt.X('k_k_fw_stove_6:O', scale=alt.Scale(paddingInner=0), title = " "),
    alt.Y('k_k_fw_stove_7:O', scale=alt.Scale(paddingInner=0), title = "Reason for turning offer down"),
)

# Configure heatmap
heatmap = base.mark_rect().encode(
    color=alt.Color('cookstove_count:Q',
        scale=alt.Scale(scheme='blues'),
        legend=None
    )
)

# Configure text
text = base.mark_text(baseline='middle', size = 15
).transform_joinaggregate(
    count = 'count(cookstove_count)',
    groupby = ['k_k_fw_stove_6', 'k_k_fw_stove_7']
).transform_calculate(
    pct = alt.datum.cookstove_count / df.shape[0] 
).encode(
    text=alt.Text('pct:Q', format = '.2%'),
    color=alt.condition(
        alt.datum.cookstove_count < 100,
        alt.value('black'),
        alt.value('white')
    )
)

locality_options = [None, 'Urban', 'Rural']
locality_labels = ['National', 'Urban', 'Rural']

locality_dropdown =alt.binding_select(
    options= locality_options, 
    labels = locality_labels, 
    name = "Locality Breakdown  ")
locality_select=alt.selection_single(
    fields=['locality_ur'],
    bind = locality_dropdown)

tiers = [None, 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_labels = ['All Tiers', 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_dropdown=alt.binding_select(options=tiers, labels = tier_labels, name = "Electricity Tier  ")
tier_select=alt.selection_single(fields=['elc_aggr_tier'],
                                bind=tier_dropdown)

# Draw the chart
chart = (heatmap + text).properties(
    height = 500,
    width = 750,
    title = ''
).add_selection(
    tier_select
).transform_filter(
    tier_select 
).add_selection(
    locality_select
).transform_filter(
    locality_select
)

chart = chart.properties(
    title={
      "text": ["Willingness to Pay over 12 months for a Fuelwood Cookstove"],
      "color": "black",
        "font": 'Times New Roman'
    }
)

final = alt.concat(chart,
    title=alt.TitleParams(
        ['', '',  
         'This chart includes responses from 16.40% of households in the Kenya MTF survey sample (total size = 4,590).',
         'Use the dropdown filters to interact with the data.'],
        baseline='bottom',
        orient='bottom',
        anchor='start',
        font = 'Times New Roman',
        fontWeight='normal',
        fontStyle = "italic",
        fontSize=15)
)

final = final.configure(font = 'Times New Roman')
final

alt.ConcatChart(...)

In [150]:
df.shape[0] / 4590

0.16405228758169935

In [64]:
final.save('Willingness to Pay over 12 months for a Fuelwood Cookstove.html')

In [65]:
final.save('Willingness to Pay over 12 months for a Fuelwood Cookstove.json')

In [ ]:
# WTP SECTION - ASK IAN FOR HELP
# wtp_d6, wtp_d7, wtp_e7, wtp_e8

In [173]:
# Load Kenya MTF Data 

chunksize = 10

mtf_list = []
chunksize = 10
for chunk in pd.read_csv("module2.csv", encoding='latin-1', chunksize = chunksize):
    mtf_list.append(chunk)
    
module2 = pd.concat(mtf_list, axis=0)
module2 = pd.DataFrame(data = module2)

In [66]:
# d4

alt.data_transformers.disable_max_rows()
source1 = module2[module2[['d_6_wtp_6m','locality', 'elc_aggr_tier']].notnull().all(1)]

c_1 = source1['elc_aggr_tier'] != 'NaN'
c_2 = source1['d_6_wtp_6m'] != 'NaN'

source = source1[conjunction(c_1,c_2)]
df = source[['elc_aggr_tier', 'locality', 'd_6_wtp_6m']]

energy_access1 = alt.Chart(df).transform_joinaggregate(
    total = 'count(*)'
).transform_calculate(
    pct = '1 / datum.total'
).encode(
    x = alt.X('d_6_wtp_6m', axis = alt.Axis(title = 'Response')),
    y = alt.Y('sum(pct):Q', axis = alt.Axis(title = "", format = '%')),
    tooltip = alt.Tooltip('d_6_wtp_6m')
).properties(
    height = 400,
    width = 300)
     
locality_options = [None, 'Urban', 'Rural', 'Peri-urban']
locality_labels = ['National', 'Urban', 'Rural', 'Peri-Urban']

locality_dropdown=alt.binding_select(
    options= locality_options, 
    labels = locality_labels, 
    name = "Locality Breakdown  ")
locality_select=alt.selection_single(
    fields=['locality'],
    bind=locality_dropdown)

tiers = [None, 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_labels = ['All Tiers', 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_dropdown=alt.binding_select(options=tiers, labels = tier_labels, name = "Electricity Tier  ")
tier_select=alt.selection_single(fields=['elc_aggr_tier'],
                                bind=tier_dropdown)

wtp_d6 = (energy_access1.mark_bar(color = '#304ca2') + energy_access1.mark_text(
    align = 'center', baseline='bottom', size = 15
).encode(
    text = alt.Text('sum(pct):Q', format = '.2%'))
).add_selection(
    locality_select
).transform_filter(
    locality_select
).add_selection(
    tier_select
).transform_filter(
    tier_select 
)

# Draw the chart
chart = (wtp_d6).properties(
    height = 500,
    width = 800,
    title = ''
).add_selection(
    tier_select
).transform_filter(
    tier_select 
).add_selection(
    locality_select
).transform_filter(
    locality_select
)

chart = chart.properties(
    title={
      "text": ["Imagine you were offered an electricity connection", 
               "at this price today and you were given 6 months to",
              "complete the payment.",
              '',
              'Would you accept the offer?'],
      "color": "black",
        "font": 'Times New Roman'
    }
)

final = alt.concat(chart,
    title=alt.TitleParams(
        ['', '',  
         'This chart includes responses from 20.23% of households in the Kenya MTF survey sample (total size = 4,590).',
         'Use the dropdown filters to interact with the data.'],
        baseline='bottom',
        orient='bottom',
        anchor='start',
        font = 'Times New Roman',
        fontWeight='normal',
        fontStyle = "italic",
        fontSize=15)
)

final = final.configure(font = 'Times New Roman')

final


alt.ConcatChart(...)

In [67]:
final.save('Willingness to Pay for Electricity Over 6 Months.html')

In [164]:
df.shape[0] / 4590

0.20239651416122004

In [68]:
alt.data_transformers.disable_max_rows()
source1 = module2[module2[['d_7_reason','locality', 'elc_aggr_tier']].notnull().all(1)]

c_1 = source1['elc_aggr_tier'] != 'NaN'
c_2 = source1['d_7_reason'] != 'NaN'

source = source1[conjunction(c_1,c_2)]
df = source[['elc_aggr_tier', 'locality', 'd_7_reason']]

energy_access1 = alt.Chart(df).transform_joinaggregate(
    total = 'count(*)'
).transform_calculate(
    pct = '1 / datum.total'
).encode(
    x = alt.X('d_7_reason',
              sort=alt.EncodingSortField(field="d_7_reason", 
                                         op="count", 
                                         order='descending'),
             axis = alt.Axis(title = "Response")),
    y = alt.Y('sum(pct):Q', axis = alt.Axis(title = "", format = '%')),
    tooltip = alt.Tooltip('d_7_reason')
).properties(
    height = 400,
    width = 600)
     
locality_options = [None, 'Urban', 'Rural', 'Peri-urban']
locality_labels = ['National', 'Urban', 'Rural', 'Peri-Urban']

locality_dropdown=alt.binding_select(
    options= locality_options, 
    labels = locality_labels, 
    name = "Locality Breakdown  ")
locality_select=alt.selection_single(
    fields=['locality'],
    bind=locality_dropdown)

tiers = [None, 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_labels = ['All Tiers', 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_dropdown=alt.binding_select(options=tiers, labels = tier_labels, name = "Electricity Tier  ")
tier_select=alt.selection_single(fields=['elc_aggr_tier'],
                                bind=tier_dropdown)

wtp_d7 = (energy_access1.mark_bar(color = '#304ca2') + energy_access1.mark_text(
    align = 'center', baseline='bottom'
).encode(
    text = alt.Text('sum(pct):Q', format = '.2%'))
).add_selection(
    locality_select
).transform_filter(
    locality_select
).add_selection(
    tier_select
).transform_filter(
    tier_select 
)

chart = chart.properties(
    title={
      "text": ["Why would you not accept the offer to pay the price", 
               "over the course of 6 months?"],
      "color": "black",
        "font": 'Times New Roman'
    }
)

final = alt.concat(chart,
    title=alt.TitleParams(
        ['', '',  
         'This chart includes responses from 18.15% of households in the Kenya MTF survey sample (total size = 4,590).',
         'Use the dropdown filters to interact with the data.'],
        baseline='bottom',
        orient='bottom',
        anchor='start',
        font = 'Times New Roman',
        fontWeight='normal',
        fontStyle = "italic",
        fontSize=15)
)

final = final.configure(font = 'Times New Roman')

final

alt.ConcatChart(...)

In [69]:
final.save('Unwillingness to Pay for Electricity Over 6 Months.html')

In [176]:
df.shape[0] /4590

0.1814814814814815

In [70]:
# d4

alt.data_transformers.disable_max_rows()
source1 = module2[module2[['e_e_7','locality', 'elc_aggr_tier']].notnull().all(1)]

c_1 = source1['elc_aggr_tier'] != 'NaN'
c_2 = source1['e_e_7'] != 'NaN'

source = source1[conjunction(c_1,c_2)]
df = source[['elc_aggr_tier', 'locality', 'e_e_7']]

energy_access1 = alt.Chart(df).transform_joinaggregate(
    total = 'count(*)'
).transform_calculate(
    pct = '1 / datum.total'
).encode(
    x = alt.X('e_e_7', 
            sort=alt.EncodingSortField(field="e_e_7", 
                                       op="count", 
                                       order='descending'),
              axis = alt.Axis(title = ' ')),
    y = alt.Y('sum(pct):Q', axis = alt.Axis(title = "", format = '%')),
    tooltip = alt.Tooltip('e_e_7')
).properties(
    height = 400,
    width = 600)
     
locality_options = [None, 'Urban', 'Rural', 'Peri-urban']
locality_labels = ['National', 'Urban', 'Rural', 'Peri-Urban']

locality_dropdown=alt.binding_select(
    options= locality_options, 
    labels = locality_labels, 
    name = "Locality Breakdown  ")
locality_select=alt.selection_single(
    fields=['locality'],
    bind=locality_dropdown)

tiers = [None, 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4']
tier_labels = ['All Tiers', 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4']
tier_dropdown=alt.binding_select(options=tiers, labels = tier_labels, name = "Electricity Tier  ")
tier_select=alt.selection_single(fields=['elc_aggr_tier'],
                                bind=tier_dropdown)

wtp_e7 = (energy_access1.mark_bar(color = '#304ca2') + energy_access1.mark_text(
    align = 'center', baseline='bottom', size = 15
).encode(
    text = alt.Text('sum(pct):Q', format = '.2%'))
).add_selection(
    locality_select
).transform_filter(
    locality_select
).add_selection(
    tier_select
).transform_filter(
    tier_select 
)


chart = wtp_e7.properties(
    title={
      "text": ["Imagine you were offered an electricity connection at this price today and",
               "you were given 12 months to complete the payment.",
              '',
              'Would you accept the offer?'],
      "color": "black",
        "font": 'Times New Roman'
    }
)

final = alt.concat(chart,
    title=alt.TitleParams(
        ['', '',  
         'This chart includes responses from 25.99% of households in the Kenya MTF survey sample (total size = 4,590).',
         'Use the dropdown filters to interact with the data.'],
        baseline='bottom',
        orient='bottom',
        anchor='start',
        font = 'Times New Roman',
        fontWeight='normal',
        fontStyle = "italic",
        fontSize=15)
)

final = final.configure(font = 'Times New Roman')

final

alt.ConcatChart(...)

In [71]:
final.save('Willingness to Pay for Electricity Over 12 Months.html')

In [179]:
df.shape[0] / 4590

0.2599128540305011

In [72]:
alt.data_transformers.disable_max_rows()
source1 = module2[module2[['e_e_8','locality', 'elc_aggr_tier']].notnull().all(1)]

c_1 = source1['elc_aggr_tier'] != 'NaN'
c_2 = source1['e_e_8'] != 'NaN'

source = source1[conjunction(c_1,c_2)]
df = source[['elc_aggr_tier', 'locality', 'e_e_8']]

energy_access1 = alt.Chart(df).transform_joinaggregate(
    total = 'count(*)'
).transform_calculate(
    pct = '1 / datum.total'
).encode(
    x = alt.X('e_e_8',
              sort=alt.EncodingSortField(field="e_e_8", 
                                         op="count", 
                                         order='descending'),
             axis = alt.Axis(title = " ")),
    y = alt.Y('sum(pct):Q', axis = alt.Axis(title = "", format = '%')),
    tooltip = alt.Tooltip('e_e_8')
).properties(
    height = 400,
    width = 600)
     
locality_options = [None, 'Urban', 'Rural', 'Peri-urban']
locality_labels = ['National', 'Urban', 'Rural', 'Peri-Urban']

locality_dropdown=alt.binding_select(
    options= locality_options, 
    labels = locality_labels, 
    name = "Locality Breakdown  ")
locality_select=alt.selection_single(
    fields=['locality'],
    bind=locality_dropdown)

tiers = [None, 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4']
tier_labels = ['All Tiers', 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4']
tier_dropdown=alt.binding_select(options=tiers, labels = tier_labels, name = "Electricity Tier ")
tier_select=alt.selection_single(fields=['elc_aggr_tier'],
                                bind=tier_dropdown)

wtp_e8 = (energy_access1.mark_bar(color = '#304ca2') + energy_access1.mark_text(
    align = 'center', baseline='bottom', size = 15
).encode(
    text = alt.Text('sum(pct):Q', format = '.2%'))
).add_selection(
    locality_select
).transform_filter(
    locality_select
).add_selection(
    tier_select
).transform_filter(
    tier_select 
)

chart = wtp_e8.properties(
    title={
      "text": ["Why would you not accept the offer to pay the price", 
               "over the course of 12 months?"],
      "color": "black",
        "font": 'Times New Roman'
    }
)

final = alt.concat(chart,
    title=alt.TitleParams(
        ['', '',  
         'This chart includes responses from 21.01% of households in the Kenya MTF survey sample (total size = 4,590).',
         'Use the dropdown filters to interact with the data.'],
        baseline='bottom',
        orient='bottom',
        anchor='start',
        font = 'Times New Roman',
        fontWeight='normal',
        fontStyle = "italic",
        fontSize=15)
)

final = final.configure(font = 'Times New Roman')

final

alt.ConcatChart(...)

In [185]:
df.shape[0] / 4590

0.21002178649237474

In [73]:
final.save('Unwillingness to Pay for Electricity Over 12 Months.html')